In [45]:
src_dir = '/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/data/derivatives'

In [48]:
import os
import csv
import numpy as np
import nibabel as nib
import dipy.tracking.utils as tu

In [49]:
def load_csv(src_csv):
    csv_fd = open(src_csv)
    csv_reader = csv.reader(csv_fd)
    csv_header = next(csv_reader)
    csv_rows = []
    for row in csv_reader:
        csv_rows.append(row)
    csv_fd.close()
    return csv_header, csv_rows

In [68]:
def split_3d_points_by_roi(src, out, tag='cortical'):
    hdr, row = load_csv(src)
    roi_split = []
    rois = list(set([r[1] for r in row]))
    xyz = {}
    for roi in rois: 
        xyz[roi]=[]
    for r in row:
        xyz[r[1]].append([r[2],r[3],r[4],r[5]])
    for roi in rois:
        roi_split.append(os.path.join(out, 'des-%s_roi-%s%s.csv' % (tag, roi)))
        with open(roi_split[-1], 'w') as out_csv:
            for des in xyz[roi]:
                out_csv.write(",".join(des) + "\n")
            print("%3d %s" % (len(xyz[roi]), roi))
    return roi_split, rois

In [173]:
def split_3d_points_by_hemi(src, out, emi='lh'):
    hdr, row = load_csv(src)
    xyz = []
    for r in row:
        if (emi == 'lh' and float(r[0]) <= 0):
            xyz.append([r[0],r[1],r[2],r[3]])
        if (emi == 'rh' and float(r[0]) >= 0):
            xyz.append([r[0],r[1],r[2],r[3]])
    with open(out, 'w') as out_csv:
        for des in xyz:
            out_csv.write(",".join(des) + "\n")
        print("Selected points: %3d/%3d" % (len(xyz), len(row)))

In [69]:
ref_tag = 'cortical_batch-03'
src_csv = os.path.join(src_dir, "des_csv/des-%s.csv" % ref_tag)
out_dir = os.path.join(src_dir, "des_csv")
_, roi_b03 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

325 MOVEMENT_ARREST
 79 AMODAL_ANOMIA
 49 MENTALIZING
 10 VISUO_SPATIAL
257 VERBAL_APRAXIA
109 ANOMIA
 70 PHONOLOGICAL
280 MOTOR
  7 VISUAL
161 SENSORIAL
  8 EYES_MOVEMENTS
  1 READING
333 SPEECH_ARREST
 47 SEMANTIC


In [70]:
ref_tag = 'cortical_batch-02'
src_csv = os.path.join(src_dir, "des_csv/des-%s.csv" % ref_tag)
out_dir = os.path.join(src_dir, "des_csv")
_, roi_b02 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

 89 MOVEMENT_ARREST
 17 AMODAL_ANOMIA
 12 MENTALIZING
  7 SPATIAL_PERCEPTION
 74 VERBAL_APRAXIA
 50 ANOMIA
  1 READING
142 MOTOR
  5 PHONOLOGICAL
 56 SENSORIAL
116 SPEECH_ARREST
 25 SEMANTIC
  3 EYES_MOVEMENT


In [71]:
ref_tag = 'cortical_batch-01'
src_csv = os.path.join(src_dir, "des_csv/des-%s.csv" % ref_tag)
out_dir = os.path.join(src_dir, "des_csv")
_, roi_b01 = split_3d_points_by_roi(src_csv, out_dir, ref_tag)

 16 SPATIAL_PERCEPTION
 68 VERBAL_APRAXIA
 44 ANOMIA
  9 PHONOLOGICAL
183 MOTOR
 94 SENSORIAL
  1 READING
135 SPEECH_ARREST
 14 SEMANTIC
  1 EYES_MOVEMENT


In [96]:
ref_tag = "ANOMIA"
b01_src = os.path.join(src_dir, "des_csv/des-cortical_batch-01_roi-%s.csv" % ref_tag)
b02_src = os.path.join(src_dir, "des_csv/des-cortical_batch-02_roi-%s.csv" % ref_tag)
b03_src = os.path.join(src_dir, "des_csv/des-cortical_batch-03_roi-%s.csv" % ref_tag)
csv_out = os.path.join(src_dir, "des_csv/des-cortical_roi-%s.csv" % ref_tag)
cmd = "cat \'%s\' \'%s\' \'%s\' >> \'%s\'" % (b01_src, b02_src, b03_src, csv_out)
os.system(cmd)


0

In [174]:
ref_tag = "ANOMIA"
ref_des = 'cortical'
ref_emi = 'lh'
src_csv = os.path.join(src_dir, "des_csv/des-%s_roi-%s.csv" % (ref_des, ref_tag))
out_csv = os.path.join(src_dir, "des_csv/des-%s_roi-%s_%s.csv" % (ref_des, ref_tag, ref_emi))
split_3d_points_by_hemi(src_csv, out_csv, ref_emi)

Selected points: 192/202


In [148]:
def cloud2nifti(src, ref, out, emi=''):
    
    img = nib.load(ref)
    vox_size = img.header.get_zooms()
    shape = img.header.get_data_shape()
    affine = img.affine
    header = img.header
    linT, offset = tu._mapping_to_voxel(affine)
    
    cloud = np.loadtxt(src, delimiter=',')
    
    mask = np.zeros(shape, dtype=np.float)
    stem = tu._to_voxel_coordinates(cloud[:,:3], linT, offset)
    i, j, k = stem.T
    if emi == 'lh':
        mask[i>90, j, k] = 1.0
    if emi == 'lh':
        mask[i<92, j, k] = 1.0
    if emi == '':
        mask[i, j, k] = 1.0
    
    nii = nib.Nifti1Image(mask, affine, header)
    nii.update_header()
    nib.save(nii, out)
    
    return stem

In [149]:
lab = 'ANOMIA'
emi = 'lh'
src = '../data/derivatives/des_csv/des-cortical_roi-%s.csv' % lab
ref = '../metadata/MNI152/MNI152_T1_brain.nii.gz'
out = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)


In [150]:
cloud2nifti(src, ref, out)

/var/folders/zc/k61jsc1n7pg31zpct00n912h0000gn/T/ipykernel_43877/4147341044.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(shape, dtype=np.float)


array([[158,  97,  95],
       [ 23,  92,  94],
       [ 24,  95, 101],
       [159, 117,  67],
       [155, 128,  56],
       [161, 108,  73],
       [163, 100,  84],
       [159,  86,  84],
       [149, 147,  75],
       [149, 147,  80],
       [160,  96, 105],
       [161, 100,  82],
       [162, 117,  59],
       [159, 124,  69],
       [125, 135, 140],
       [161, 109,  71],
       [163,  77,  63],
       [163, 103,  75],
       [159,  91,  89],
       [159, 102,  73],
       [147, 144,  67],
       [157,  96, 104],
       [ 32, 108,  78],
       [ 32,  93,  85],
       [154, 140,  90],
       [160, 117,  71],
       [150, 154,  81],
       [151, 155,  70],
       [157,  99,  34],
       [161, 114,  74],
       [148, 146, 108],
       [134, 146, 127],
       [156, 130,  71],
       [160,  97,  92],
       [146, 159,  77],
       [148, 149, 104],
       [154,  90,  88],
       [150, 145,  73],
       [160,  84,  90],
       [158,  99,  79],
       [159,  83,  68],
       [ 22,  95

In [162]:
lab = 'VERBAL_APRAXIA'
emi = "lh"
src_vol = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)
out_gii = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s_sfwhm-10.gii' % (lab, emi)

In [163]:
cmd = "mri_vol2surf --mov %s --surf pial --mni152reg --out %s --hemi lh --surf-fwhm 10" % (src_vol, out_gii)
os.system(cmd)

srcvol = ../data/derivatives/point_to_surf/des-cortical_roi-VERBAL_APRAXIA_lh.nii.gz
srcreg = /Applications/freesurfer/7.1.1/average/mni152.register.dat
srcregold = 0
srcwarp unspecified
surf = pial
hemi = lh
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
niiRead(): NIFTI_UNITS_UNKNOWN, assuming mm
niiRead(): detected input as 64 bit double, reading in as 32 bit float
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
-------- original matrix -----------
 0.99753  -0.00732   0.01760   0.95709;
-0.01296  -0.00926   0.99706  -17.81596;
-0.01460  -1.00094   0.00244  -18.54964;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Applications/freesurfer/7.1.1/subjects/fsaverage/surf/lh.pial
Done reading source surface
Mapping Source Volume onto Source Subject Surface
 1 0 0 0
using old
Done mapping volume to surface
N

0

In [120]:
lab = 'ANOMIA'
emi = "lh"
vol = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)
surf = "lh.pial.asc"
projtype = "1"
projdist = "10"
projmap = "1"
reg = "mni2reg"
vsm = "novsm"
interp = "1"
out = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.gii' % (lab, emi)
cmd = "mri_vol2surf --vol2surf %s %s %s %s %s %s %s %s %s" % \
    (vol, surf, projtype, projdist, projmap, reg, vsm, interp, out)
cmd

'mri_vol2surf --vol2surf ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.nii.gz lh.pial.asc 1 10 1 mni2reg novsm 1 ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii'

In [101]:
os.system(cmd)


srcvol = ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.nii.gz
srcreg = /Applications/freesurfer/7.1.1/average/mni152.register.dat
srcregold = 0
srcwarp unspecified
surf = pial
hemi = lh
reshape = 0
interp = nearest
float2int = round
GetProjMax = 0
INFO: float2int code = 0
niiRead(): NIFTI_UNITS_UNKNOWN, assuming mm
niiRead(): detected input as 64 bit double, reading in as 32 bit float
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
-------- original matrix -----------
 0.99753  -0.00732   0.01760   0.95709;
-0.01296  -0.00926   0.99706  -17.81596;
-0.01460  -1.00094   0.00244  -18.54964;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
INFO: smoothing volume at fwhm = 20 mm (std = 8.49322)
Reading surface /Applications/freesurfer/7.1.1/subjects/fsaverage/surf/lh.pial
Done reading source surface
Mapping Source Volume onto Source Subject Surface
 1 0

0

In [102]:
lab = 'ANOMIA'
emi = "lh"
ref_surf = '/Volumes/paolo/Downloads/point_to_surf/lh.pial.asc'
src_vol = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.nii.gz' % (lab, emi)
out_gii = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s_fsl.gii' % (lab, emi)

In [103]:
cmd = "surf_proj  --data=%s --surf=%s --out=%s --surfout" % (src_vol, ref_surf, out_gii)

In [104]:
os.system(cmd)

read data
read surface
project data
save
numda2 3
write vertcies 163842
write traingles 327680
327680 163842
resize trinagle array
write faces 983040
scalar 0 2
set scalar names 1 MyScalars
do memcpy 
scalar int 0
done scalar
null


0

In [114]:
lab = 'ANOMIA'
emi = "lh"
src_gii = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.gii' % (lab, emi)
out_stl = '../data/derivatives/point_to_surf/des-cortical_roi-%s_%s.stl' % (lab, emi)

In [115]:
cmd = "mris_convert %s %s" % (src_gii, out_stl)
cmd

'mris_convert ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.stl'

In [116]:
os.system(cmd)

mriseadGIFTIfile: mris is NULL! found when parsing file ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii
error: Invalid argument
error: mris_convert: could not read surface file ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii 


65280

In [156]:
%cd '/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/preprocessing'

/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/preprocessing


In [157]:
pwd

'/Volumes/GoogleDrive/My Drive/Projects/NeuSurPlan_2021/IntraOpMap/code/preprocessing'

In [83]:
ls ../data/derivatives/point_to_surf

MNI152_T1_1mm_brain.stl*
MNI152_T1_1mm_brain_lh.gii*
MNI152_T1_1mm_brain_lh.stl*
MNI152_T1_1mm_brain_lh_hole.ply*
des-cortical_batch-03_roi-MOTOR_lh.csv*
des-cortical_batch-03_roi-MOTOR_lh.gii*
des-cortical_batch-03_roi-MOTOR_lh.nii.gz*
des-cortical_batch-03_roi-MOTOR_lh.ply*
des-cortical_batch-03_roi-MOTOR_lh_fwhm-10.gii*
des-cortical_batch-03_roi-MOTOR_lh_fwhm-20.gii*
des-cortical_roi-SPEECH_ARREST_lh.nii.gz*
des-cortical_roi-VERBAL_APRAXIA_lh.nii.gz*
orig.nii.gz*
rh.MNI152_T1_1mm_brain.gii*


In [112]:
ls ../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii

../data/derivatives/point_to_surf/des-cortical_roi-ANOMIA_lh.gii*


In [132]:
img = nib.load('../metadata/MNI152/MNI152_T1_1mm.nii.gz')


In [141]:
img.header.set_xyzt_units("mm", "sec")

In [134]:
img.header.get_dim_info()

(None, None, None)

In [135]:
img.header.get_data_dtype()

dtype('<f4')

In [142]:
img.header.get_xyzt_units()

('mm', 'sec')

AttributeError: 'tuple' object has no attribute 'dtype'

In [147]:
nib.Nifti1Image?

In [151]:
import dipy.stats.analysis as dsa

In [152]:
dsa.gaussian_weights?

In [168]:
int("31")

31